# 1. LOAD DeepSeek

In [1]:
# ref: https://gist.github.com/vgel/8a2497dc45b1ded33287fa7bb6cc1adc
from transformers import AutoModelForCausalLM, AutoTokenizer, DynamicCache
import torch

device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B'
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained('/scratch/ss4yd/huggingfacemodels/DeepSeek-R1-Distill-Qwen-7B/', 
                                             torch_dtype=torch.bfloat16, device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# 2. Generate Prompt from Opinions QA dataset

In [20]:
from process_opinions_qa import generate_prompt
import random
import yaml

# read yaml file
# change survey_type and context in config.yaml file for different types of prompts as described in the paper
with open('config.yaml') as file:
    config = yaml.safe_load(file)
print(f"Context: {config['prompt']['context']} \nSurvey file: {config['prompt']['survey_type']}")

manual=True # use config file if False, set manually if True
if manual:
    config=dict()
    config['prompt']=dict()
    config['prompt']['context']='steer-qa' # values: default, steer-qa, steer-bio, steer-portray
    config['prompt']['survey_type']='Pew_American_Trends_Panel_disagreement_500' # values: Pew_American_Trends_Panel_W26, replace 26 with the [27,27,29,32..] etc.
    config['prompt']['output_path']='./'
    config['prompt']['include_output']=False
    config['prompt']['reference_index']=None
    
train, evals=generate_prompt(config)

# print(train[0])

while True:
    if len(train)>0:
        prompt=train[random.choice(range(len(train)))]+'\n'+evals[random.choice(range(len(evals)))]
    else:
        prompt=evals[random.choice(range(len(evals)))]
    
    if 'Trump' in prompt:
        break

print('\nGenerated prompt:')
print(prompt)



Context: steer-bio 
Survey file: Pew_American_Trends_Panel_disagreement_500

Generated prompt:
Question: What is your present religion, if any?
A. Protestant
B. Roman Catholic
C. Mormon
D. Orthodox
E. Jewish
F. Muslim
G. Buddhist
H. Hindu
I. Atheist
J. Agnostic
K. Other
L. Nothing in particular
M. Refused
Answer: F

Question: Since President Trump was elected, do you think it has become more acceptable or less acceptable for people to express racist or racially insensitive views, or is it about as acceptable as it was before?
A. More acceptable
B. Less acceptable
C. About as acceptable
D. Refused
Answer:


# 3. Prompt model with chain-of-thought prompting

Taken from: https://gist.github.com/vgel/8a2497dc45b1ded33287fa7bb6cc1adc

Code forces the model to generate at least 128 tokens before stopping. The code also used replacement tokens to bridge this.

In [21]:
# ref: https://gist.github.com/vgel/8a2497dc45b1ded33287fa7bb6cc1adc

_, _start_think_token, end_think_token = tokenizer.encode("<think></think>")

replacements=["\nWait, but", "\nHmm", "\nSo"]
@torch.inference_mode
def reasoning_effort(question: str, min_thinking_tokens: int):
    tokens = tokenizer.apply_chat_template(
        [
            {"role": "user", "content": question},
            {"role": "assistant", "content": "<think>\n" + ""},
        ],
        continue_final_message=True,
        return_tensors="pt",
    )
    tokens = tokens.to(model.device)
    kv = DynamicCache()
    n_thinking_tokens = 0

    print(tokenizer.decode(list(tokens[0])))
    while True:
        out = model(input_ids=tokens, past_key_values=kv, use_cache=True)
        next_token = torch.multinomial(
            torch.softmax(out.logits[0, -1, :], dim=-1), 1
        ).item()
        kv = out.past_key_values

        if (
            next_token in (end_think_token, model.config.eos_token_id)
            and n_thinking_tokens < min_thinking_tokens
        ):
            replacement = random.choice(replacements)
            print(replacement)
            replacement_tokens = tokenizer.encode(replacement)
            n_thinking_tokens += len(replacement_tokens)
            tokens = torch.tensor([replacement_tokens]).to(tokens.device)
        elif next_token == model.config.eos_token_id:
            break
        else:
            yield tokenizer.decode([next_token])
            n_thinking_tokens += 1
            tokens = torch.tensor([[next_token]]).to(tokens.device)

## 3a. Run model with CoT prompting

In [22]:
for chunk in reasoning_effort(prompt, 128):
    print(chunk, end="", flush=True)

<｜begin▁of▁sentence｜><｜User｜>Question: What is your present religion, if any?
A. Protestant
B. Roman Catholic
C. Mormon
D. Orthodox
E. Jewish
F. Muslim
G. Buddhist
H. Hindu
I. Atheist
J. Agnostic
K. Other
L. Nothing in particular
M. Refused
Answer: F

Question: Since President Trump was elected, do you think it has become more acceptable or less acceptable for people to express racist or racially insensitive views, or is it about as acceptable as it was before?
A. More acceptable
B. Less acceptable
C. About as acceptable
D. Refused
Answer:<｜Assistant｜><think>

Okay, I'm trying to figure out what the answer is to the question about President Trump's impact on expressing racist views. The options are A. More acceptable, B. Less acceptable, C. About as acceptable, D. Refused. But there's no answer provided yet.

So, I know that President Trump, especially during his time as the Republican candidate and then as the POTUS, was a big figure in making race and racism a topic of national conve

# 3b Run model without CoT prompting

This code was adapted from https://github.com/stanford-crfm/helm cited in the opinions-qa github repo to reproduce their results. It forces the model to generate only one token, which is usually one of the options presented in the prompt.

Ref:https://github.com/stanford-crfm/helm/blob/main/src/helm/clients/huggingface_client.py

In [23]:
stopping_criteria = None
raw_request={
    "prompt":prompt,
    "stop_sequences": [],
    "temperature":1e-7,
    "max_new_tokens":1,
    "top_p":1,
    "num_return_sequences":1,
    "echo_prompt":False
}
raw_request

{'prompt': 'Question: What is your present religion, if any?\nA. Protestant\nB. Roman Catholic\nC. Mormon\nD. Orthodox\nE. Jewish\nF. Muslim\nG. Buddhist\nH. Hindu\nI. Atheist\nJ. Agnostic\nK. Other\nL. Nothing in particular\nM. Refused\nAnswer: F\n\nQuestion: Since President Trump was elected, do you think it has become more acceptable or less acceptable for people to express racist or racially insensitive views, or is it about as acceptable as it was before?\nA. More acceptable\nB. Less acceptable\nC. About as acceptable\nD. Refused\nAnswer:',
 'stop_sequences': [],
 'temperature': 1e-07,
 'max_new_tokens': 1,
 'top_p': 1,
 'num_return_sequences': 1,
 'echo_prompt': False}

In [24]:
encoded_input = tokenizer(raw_request["prompt"], return_tensors="pt", return_token_type_ids=False).to(device)

output = model.generate(
                **encoded_input,
#                 temperature=raw_request["temperature"],
                num_return_sequences=raw_request["num_return_sequences"],
                max_new_tokens=raw_request["max_new_tokens"],
                top_p=raw_request["top_p"],
                do_sample=False,
                return_dict_in_generate=True,
                output_scores=True,
                stopping_criteria=stopping_criteria,
            )
sequences = output.sequences
scores = output.scores

print(tokenizer.decode(sequences[0]))

# calculate log probs when scores do not return inf
if torch.isinf(scores[0]).sum().item()==0:

    # Compute logprobs of generated tokens for each completed sequence.
    all_generated_tokens_logprobs = []
    for completion_id in range(raw_request["num_return_sequences"]):
        generated_tokens_logprobs = []
        for i in range(len(sequences[completion_id]) - len(encoded_input.input_ids[0])):
            logprobs = torch.nn.functional.log_softmax(scores[i][completion_id], dim=0)
            # Get log probability of chosen token.
            j = i + len(encoded_input.input_ids[0])
            generated_tokens_logprobs.append(logprobs[sequences[completion_id][j]].item())
        all_generated_tokens_logprobs.append(generated_tokens_logprobs)

    print(f'Log Probs: {all_generated_tokens_logprobs}')
else:
    print('Log Probs not calculated')

/home/ss4yd/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜>Question: What is your present religion, if any?
A. Protestant
B. Roman Catholic
C. Mormon
D. Orthodox
E. Jewish
F. Muslim
G. Buddhist
H. Hindu
I. Atheist
J. Agnostic
K. Other
L. Nothing in particular
M. Refused
Answer: F

Question: Since President Trump was elected, do you think it has become more acceptable or less acceptable for people to express racist or racially insensitive views, or is it about as acceptable as it was before?
A. More acceptable
B. Less acceptable
C. About as acceptable
D. Refused
Answer: B
Log Probs: [[-0.3152509331703186]]


In [12]:
# Remove prompt from the start of each sequence if echo_prompt is False.
if not raw_request["echo_prompt"]:
    sequences = [sequence[len(encoded_input.input_ids[0]) :] for sequence in sequences]

In [13]:
all_tokens = [[tokenizer.decode(token) for token in sequence_tokens] for sequence_tokens in sequences]
all_decoded_text = tokenizer.batch_decode(sequences)

In [14]:
torch.isinf(scores[0]).sum().item()

0